In [3]:
import random
import time
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import defaultdict

# Cấu hình trình duyệt với Undetected ChromeDriver
options = uc.ChromeOptions()
options.add_argument("--incognito")  # Chạy ẩn danh
options.add_argument("--disable-blink-features=AutomationControlled")  # Ẩn Selenium
options.add_argument("start-maximized")  # Mở full màn hình
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")  # Giả lập trình duyệt


In [4]:
driver = uc.Chrome(options=options)
driver.get("https://www.thegioididong.com/laptop")

# Chờ tải trang
time.sleep(random.uniform(4, 7))

# Lấy danh sách sản phẩm bằng cách bấm nút "Xem thêm"
while True:
    try:
        button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "see-more-btn"))
        )
        ActionChains(driver).move_to_element(button).perform()
        button.click()
        time.sleep(random.uniform(3, 6))  # Tăng thời gian delay ngẫu nhiên
    except:
        print("Không còn nút 'Xem thêm', đã tải hết sản phẩm.")
        break


Không còn nút 'Xem thêm', đã tải hết sản phẩm.


In [9]:
data = defaultdict(list)

def click_and_get_specs(group_ids):
    """Lấy thông số kỹ thuật bằng cách click vào các nhóm thông số"""
    specs_data = {}

    for group_id in group_ids:
        try:
            button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, f'a[data-group-id="{group_id}"]'))
            )
            ActionChains(driver).move_to_element(button).perform()
            driver.execute_script("arguments[0].click();", button)

            WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".text-specifi.active li"))
            )
            time.sleep(0.5)
        except:
            continue

        specs = driver.find_elements(By.CSS_SELECTOR, ".text-specifi.active li")
        max_specs_length = 6
        specs_data[group_id] = ([spec.text for spec in specs] + ["Không có"] * max_specs_length)[:max_specs_length]

    return specs_data

def fetch_product_data(link):
    global driver, data

    if not link.startswith("http"):
        print(f"⚠ Bỏ qua link không hợp lệ: {link}")
        return
    
    try:
        driver.get(link)
        time.sleep(random.uniform(4, 7))

        # Lấy thông tin RAM & ổ cứng từ product_id = 34
        specs_data = click_and_get_specs([32, 62])

        # Thêm dữ liệu vào danh sách
        data["Link"].append(link)
        data["Kich_thuoc"].append(specs_data.get(62, ["Không có"])[0])
        data["Chat_lieu"].append(specs_data.get(62, ["Không có"])[1])
        data["He_thong_pin"].append(specs_data.get(62, ["Không có"])[2])
        data["He_dieu_hanh"].append(specs_data.get(62, ["Không có"])[3])
        data["Thoi_gian_ra_mat"].append(specs_data.get(62, ["Không có"])[4])

        print(f"✔ Đã lấy dữ liệu sản phẩm từ: {link}")
    except Exception as e:
        print(f"❌ Lỗi khi lấy sản phẩm {link}: {str(e)}")

        # Đảm bảo dữ liệu có đủ cột
        data["Link"].append(link)
        data["Kich_thuoc"].append("Không có")
        data["Chat_lieu"].append("Không có")
        data["He_thong_pin"].append("Không có")
        data["He_dieu_hanh"].append("Không có")
        data["Thoi_gian_ra_mat"].append("Không có")


# Lấy danh sách URL sản phẩm
try:
    product_urls = list(set([
        item.get_attribute("href") for item in driver.find_elements(By.CSS_SELECTOR, ".listproduct a[href]")
        if item.get_attribute("href").startswith("http")
    ]))
except:
    print("Lỗi khi lấy danh sách sản phẩm.")
    driver.quit()
    exit()

if not product_urls:
    print("Không tìm thấy sản phẩm nào.")
    driver.quit()
    exit()

# Lấy thông tin từ từng sản phẩm
for link in product_urls:
    fetch_product_data(link)

# Đảm bảo tất cả danh sách có cùng độ dài
max_length = max(len(v) for v in data.values())

for key in data:
    while len(data[key]) < max_length:
        data[key].append("Không có")

# Xuất dữ liệu ra file CSV
df = pd.DataFrame(data)
df.to_csv("chatlieu.csv", index=False, encoding="utf-8-sig")
print("📂 Dữ liệu đã được lưu vào cpu_specs.csv")

# Đóng trình duyệt
driver.quit()

✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/msi-bravo-15-b7ed-r5-010vn
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/msi-cyborg-15-a13vek-i7-1423vn
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/asus-rog-strix-g16-g614jv-i7-n3515w
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/asus-vivobook-x1404za-i3-nk246w
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/hp-elitebook-635-aero-g11-r5-a8ws6pt
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/dell-latitude-5450-ultra-5-l5450125u16512w
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/hp-envy-x360-14-fc0162tu-ultra-5-ay8x1pa
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/msi-gaming-gf63-thin-12uc-i7-887vn
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/msi-katana-a15-ai-b8vg-r9-466vn
✔ Đã lấy dữ liệu sản phẩm từ: https://www.thegioididong.com/laptop/dell-inspiron-15